# 基差-库存-利润分析

## 数据更新和加载

In [2]:
import pandas as pd
import akshare as ak
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import datetime, date

## 基差分析

### 基差走势

In [3]:
# 加载基差数据（奇货可居网）
df_rb_basis = pd.read_csv('data/rb-main-basis-20231028.csv')
df_rb_basis['基差'] = df_rb_basis['现货'] - df_rb_basis['期货']
# 加载库存数据（AK-Share网站）
futures_inventory_em_df = ak.futures_inventory_em(symbol="螺纹钢")
futures_inventory_em_df['日期'] = pd.to_datetime(futures_inventory_em_df['日期'], format="%Y-%m-%d")
# 统一日期格式
df_rb_basis['日期'] = pd.to_datetime(df_rb_basis['日期'], format="%Y-%m-%d")
# 合并基差和库存数据
df_rb0 = pd.merge(df_rb_basis, futures_inventory_em_df, on='日期', how='outer')
# 空数据清理
df_rb0 = df_rb0.fillna(0)
# 为基差率（正负值）标记颜色
df_rb0['基差率颜色'] = df_rb0['基差率'] > 0
df_rb0['基差率颜色'] = df_rb0['基差率颜色'].replace({True:1, False:0})
max_y = df_rb0['期货'].max() * 1.05
min_y = df_rb0['期货'].min() * 0.95

fig = make_subplots(rows=3, cols=1, shared_xaxes=True, specs=[[{"secondary_y": True}], [{"secondary_y": True}], [{"secondary_y": True}]],
                   vertical_spacing=0.01, subplot_titles=('基差分析', '基差率', '库存'), 
                   row_width=[0.15, 0.15, 0.7])

#fig_main = make_subplots(specs=[[{"secondary_y": True}]])
fig_future_price = go.Scatter(x=df_rb0['日期'], y=df_rb0['期货'], name='期货价格', marker_color='rgb(84,134,240)')
fig_spot_price = go.Scatter(x=df_rb0['日期'], y=df_rb0['现货'], name='现货价格', marker_color='rgb(105,206,159)')
fig_basis = go.Scatter(x=df_rb0['日期'], y=df_rb0['基差'], stackgroup='one', name='基差', marker_color='rgb(239,181,59)')
#fig_main.add_trace(fig_future_price)
#fig_main.add_trace(fig_basis_rate, secondary_y=True)

fig.add_trace(fig_basis, secondary_y=True)
fig.add_trace(fig_future_price, row = 1, col = 1)
fig.add_trace(fig_spot_price, row = 1, col = 1)

fig_basis_rate = go.Bar(x=df_rb0['日期'], y = df_rb0['基差率'], name='基差率', marker_color='rgb(244,128,68)')
#fig_basis_rate = go.Bar()
#fig_basis_rate.mode = 'markers'
# fig_basis_rate.x = df_rb0['日期']
# fig_basis_rate.y = df_rb0['基差率']
fig_basis_rate.marker.colorscale = ['green', 'red']
fig_basis_rate.marker.color = df_rb0['基差率颜色']
#fig_basis_rate.marker.size = 2
fig_basis_rate.marker.showscale = False
fig_basis_rate.showlegend = False

fig.add_trace(fig_basis_rate, row = 2, col = 1)
fig_storage = go.Bar(x=df_rb0['日期'], y=df_rb0['库存'], name='库存', marker_color='rgb(234,69,70)')
#fig_storage = px.bar(df_rb0, x='日期', y='库存')
fig.add_trace(fig_storage, row = 3, col = 1)

trade_date = ak.tool_trade_date_hist_sina()['trade_date']
trade_date = [d.strftime("%Y-%m-%d") for d in trade_date]
dt_all = pd.date_range(start=df_rb0['日期'].iloc[0],end=df_rb0['日期'].iloc[-1])
dt_all = [d.strftime("%Y-%m-%d") for d in dt_all]
dt_breaks = list(set(dt_all) - set(trade_date))

# X轴坐标按照年-月显示
fig.update_xaxes(
    showgrid=True,
    zeroline=True,
    showline=True,
    dtick="M1",  # 按月显示
    ticklabelmode="period",   # instant  period
    tickformat="%b\n%Y",
    rangebreaks=[dict(values=dt_breaks)],
    rangeslider_visible = False, # 下方滑动条缩放
    rangeselector = dict(
        # 增加固定范围选择
        buttons = list([
            dict(count = 1, label = '1M', step = 'month', stepmode = 'backward'),
            dict(count = 6, label = '6M', step = 'month', stepmode = 'backward'),
            dict(count = 1, label = '1Y', step = 'year', stepmode = 'backward'),
            dict(count = 1, label = 'YTD', step = 'year', stepmode = 'todate'),
            dict(step = 'all')
            ]))
)
#fig.update_traces(xbins_size="M1")
fig.update_layout(
    yaxis_range=[min_y,max_y],
    #autosize=False,
    #width=800,
    height=800,
    margin=dict(l=0, r=0, t=0, b=0)
)

fig.show()

## 基差率分析

### 基差率季节分析

In [4]:
df_rb0['年度'] = df_rb0['日期'].dt.year
df_rb0['年内日期'] = df_rb0['日期'].dt.strftime('1900-%m-%d')
fig_basis_rate_season = px.line(df_rb0,
                                x='年内日期',
                                y='基差率',
                                color='年度',
                                #color_discrete_sequence=px.colors.qualitative.G10)
                                color_discrete_sequence=['lightgray', 'lightblue', 'orange', 'red'])
fig_basis_rate_season.update_layout(
    title={
        'text':'基差率季节分析',
        'xanchor':'center'},
    margin=dict(l=10, r=10, t=40, b=10)
)

fig_basis_rate_season.show()

In [47]:
import history_data as hd
df1 = hd.load_choice_file('data/钢铁产业链/中游数据/螺纹钢/螺纹钢库存.xlsx')
df1

,日期,仓单数量：螺纹钢,库存：螺纹钢：合计,库存：螺纹钢：上海,库存：螺纹钢：杭州,库存：螺纹钢：武汉,库存：螺纹钢：广州,库存：螺纹钢：成都,库存：螺纹钢：昆明,库存：螺纹钢：北京,...,库存：螺纹钢：合计：环比,库存：螺纹钢：上海：环比,库存：螺纹钢：杭州：环比,库存：螺纹钢：武汉：环比,库存：螺纹钢：广州：环比,库存：螺纹钢：成都：环比,库存：螺纹钢：昆明：环比,库存：螺纹钢：北京：环比,库存：螺纹钢：西安：环比,库存：钢筋：当月值
0,2010-03-29 00:00:00,140971,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-03-30 00:00:00,152633,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-03-31 00:00:00,154121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,121.8983
3,2010-04-01 00:00:00,166995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010-04-02 00:00:00,178067,691.4,81.4,68.6,24.4,58.5,35.2,30.2,49.8,...,-2.055503,-1.619531,-0.507614,-5.791506,-4.878049,-0.84507,0.165837,-6.002265,-11.072664,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3396,2023-11-13 00:00:00,49680,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3397,2023-11-14 00:00:00,47854,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3398,2023-11-15 00:00:00,46652,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3399,2023-11-16 00:00:00,46652,368.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-3.079348,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 基差率月度涨跌统计

### 基差率频率分布

## 跨期分析

### 期限结构

In [6]:
# 加载合约基础数据
futures_comm_info = pd.read_excel('data/common_info.xlsx')
spec_contact_list = futures_comm_info[futures_comm_info.合约名称.str.startswith('螺纹钢')]
fig_term = make_subplots(specs=[[{"secondary_y": True}]])
fig_term.add_trace(go.Scatter(x=spec_contact_list['合约代码'], y=spec_contact_list['现价']))
# 获取最新现货价格
spot_price = df_rb0[df_rb0['现货']!=0]['现货'].iloc[-1]
fig_term.add_hline(y=spot_price)
fig_term.update_layout(
    title={
        'text':'期限结构'
    },
    #autosize=False,
    width=800,
    #height=800,
    margin=dict(l=10, r=10, t=40, b=10)
)
fig_term.show()

### 套利分析

#### 价差分析-多期排列

#### 价差分析-跨期价差矩阵

#### 基差-月差分析

#### 价差季节性分析

## 库存

### 库存数据

In [7]:
futures_inventory_em_df = ak.futures_inventory_em(symbol="螺纹钢")
futures_inventory_em_df['日期'] = pd.to_datetime(futures_inventory_em_df['日期'], format="%Y-%m-%d")
df_rb_basis['日期'] = pd.to_datetime(df_rb_basis['日期'], format="%Y-%m-%d")
df_rb0 = pd.merge(df_rb_basis, futures_inventory_em_df, on='日期', how='outer')

### 库存周期

### 仓单

#### 注册仓单

In [8]:
#future_receipt = ak.get_receipt(start_day="20230101", end_day="20231031", vars_list=["RB"])
#future_receipt

#### 仓单日报

#### 期转现

#### 交割统计

## 利润

## 综合分析